### Purpose
Take all streets in the code, categorise by type, delete duplicates

In [32]:
import re
import pandas as pd

# Configuration
INPUT_PATH = r"C:\Users\TheAiro\Desktop\Machine Learning Projects\Visuem\data\all_street_names_types.csv"

df = pd.read_csv(INPUT_PATH, sep=",", keep_default_na=False)
df = df.drop_duplicates(subset=["Name"], keep="first")

print(df.shape)

# Remove rows with city/village markers in the name
f = df[df['Name'].str.contains(r'(?:^| )шосе', regex=True, na=False)]
df = df[~df['Name'].str.contains(r'(?:^| )Шосе', regex=True, na=False)]

print(f"Dataset after filtering: {df.shape}")
print(f"Total rows removed: check previous shape")

f.head()

(39850, 3)
Dataset after filtering: (39850, 3)
Total rows removed: check previous shape


,Unnamed: 0,Name,Type


In [30]:
# Save the cleaned dataset
output_path = r"C:\Users\TheAiro\Desktop\Machine Learning Projects\Visuem\data\all_street_names_types.csv"
df.to_csv(output_path, sep=",", index=False, encoding="utf-8")
print(f"Cleaned dataset saved to: {output_path}")
print(f"Total rows saved: {len(df)}")

Cleaned dataset saved to: C:\Users\TheAiro\Desktop\Machine Learning Projects\Visuem\data\all_street_names_types.csv
Total rows saved: 39850
